In [ ]:

import os.path as osp
import random
import time

import cv2
import numpy as npS
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data

In [ ]:

torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

# 데이터 로더 만들기

VGG 모듈의 초기값에 미리 VGG 모듈을 ImageNet 화상 분류 작업으로 학습시킨 결합 파라미터를 사용한다. 

vgg의 모듈의 초깃값은 "He 초깃값"을 사용한다. He 초깃값은 활성화 함수가 ReLU일 때 사용하는 초기화 방법이다. 각 합성곱 층에서 입력 채널 수가 input_n인 경우 합성곱 층의 결합 파라미터의 초깃값으로 평균  0, 표준편차 sqrt(2/input_n)의 가우스 분포에 따른 난수를 사용한다.

In [ ]:
from utils.ssd_model import make_datapath_list, VOCDataset, DataTransform, Anno_xml2list, od_collate_fn


rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(
    rootpath)

voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']
color_mean = (104, 117, 123)  
input_size = 300 

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))


batch_size = 32

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, collate_fn=od_collate_fn)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, collate_fn=od_collate_fn)

dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


# 네트워크 모델 만들기:

네트워크 모델의 결합 파라미터 초깃값 설정을 추가하여 네트워크 모델을 만든다.

In [ ]:
from utils.ssd_model import SSD

# SSD 300으로 설정
ssd_cfg = {
    'num_classes': 21, 
    'input_size': 300, 
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4], 
    'feature_maps': [38, 19, 10, 5, 3, 1], 
    'steps': [8, 16, 32, 64, 100, 300],  
    'min_sizes': [30, 60, 111, 162, 213, 264],  
    'max_sizes': [60, 111, 162, 213, 264, 315], 
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

# SSD 네트워크 모델
net = SSD(phase="train", cfg=ssd_cfg)

#SSD의 초기 가중치 설정
# ssd의 vgg에 가중치 로드
vgg_weights = torch.load('./weights/vgg16_reducedfc.pth')
net.vgg.load_state_dict(vgg_weights)

# ssd의 기타 네트워크 가중치는 He의 초기치로 초기화
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None:  
            nn.init.constant_(m.bias, 0.0)

# He의 초기치 적용 
net.extras.apply(weights_init)
net.loc.apply(weights_init)
net.conf.apply(weights_init)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


# 손실함수 및 최적화 기법 설정

In [ ]:
from utils.ssd_model import MultiBoxLoss

# 손실 함수 설정
criterion = MultiBoxLoss(jaccard_thresh=0.5, neg_pos=3, device=device)

# 최적화 기법 설정
optimizer = optim.SGD(net.parameters(), lr=1e-3,
                      momentum=0.9, weight_decay=5e-4)


# 학습 및 검증 실시

In [ ]:


# 모델을 학습시키는 함수 작성

def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):


    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # 네트워크를 GPU로
    net.to(device)

    # 네트워크가 어느 정도 고정되면 고속화 시킨다.
    torch.backends.cudnn.benchmark = True

    # 반복자의 카운터 설정
    iteration = 1
    epoch_train_loss = 0.0 # 에폭 손실 합
    epoch_val_loss = 0.0  # 에폭 손실 합
    logs = []

    # 에폭 루프
    for epoch in range(num_epochs+1):

        # 시작 시간 저장
        t_epoch_start = time.time()
        t_iter_start = time.time()

        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

   
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  
                print('（train）')
            else:
                if((epoch+1) % 10 == 0):
                    net.eval()   
                    print('-------------')
                    print('（val）')
                else:
                   
                    continue

           
            for images, targets in dataloaders_dict[phase]:

          
                images = images.to(device)
                targets = [ann.to(device)
                           for ann in targets]  
                
                # 옵티마이저 초기화
                optimizer.zero_grad()

                # 순전파 게산
                with torch.set_grad_enabled(phase == 'train'):
                    #순전파 계산
                    outputs = net(images)

                    # 손실 계산
                    loss_l, loss_c = criterion(outputs, targets)
                    loss = loss_l + loss_c

                    # 훈련 시에는 역전파
                    if phase == 'train':
                        loss.backward() # 경사 계산 

                        # 경사가 너무 커지면 계산이 부정확해 clip에서 최대 경사 2.0에 고정
                        nn.utils.clip_grad_value_(
                            net.parameters(), clip_value=2.0)

                        optimizer.step() # 파라미터 갱신

                        if (iteration % 10 == 0):  
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('イテレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                                iteration, loss.item(), duration))
                            t_iter_start = time.time()

                        epoch_train_loss += loss.item()
                        iteration += 1

                    else:
                        epoch_val_loss += loss.item()

 
        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss, epoch_val_loss))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

       
        log_epoch = {'epoch': epoch+1,
                     'train_loss': epoch_train_loss, 'val_loss': epoch_val_loss}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv("log_output.csv")

        epoch_train_loss = 0.0  
        epoch_val_loss = 0.0  

    
        if ((epoch+1) % 10 == 0):
            torch.save(net.state_dict(), 'weights/ssd300_' +
                       str(epoch+1) + '.pth')


In [ ]:
num_epochs = 50
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs = num_epochs)